 # Covid-19 impact on Digital Learning

<img src='https://res.cloudinary.com/people-matters/image/upload/fl_immutable_cache,w_624,h_351,q_auto,f_auto/v1505193006/1505193005.jpg'>

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

### Product information data
The product file `products_info.csv` includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy. Data were labeled by our team. Some products may not have labels due to being duplicate, lack of accurate url or other reasons.

| Name | Description |
| :--- | :----------- |
| LP ID| The unique identifier of the product |
| URL | Web Link to the specific product |
| Product Name | Name of the specific product |
| Provider/Company Name | Name of the product provider |
| Sector(s) | Sector of education where the product is used |
| Primary Essential Function | The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled |


In [ ]:
products_df = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')
products_df.head()

### District information data

The district file `districts_info.csv` includes information about the characteristics of school districts, including data from [NCES](https://nces.ed.gov/) (2018-19), [FCC](https://www.fcc.gov/) (Dec 2018), and [Edunomics Lab](https://edunomicslab.org/). In this data set, we removed the identifiable information about the school districts. We also used an open source tool [ARX](https://arx.deidentifier.org/) [(Prasser et al. 2020)](https://onlinelibrary.wiley.com/doi/full/10.1002/spe.2812) to transform several data fields and reduce the risks of re-identification. For data generalization purposes some data points are released with a range where the actual value falls under. Additionally, there are many missing data marked as 'NaN' indicating that the data was suppressed to maximize anonymization of the dataset. 

| Name | Description |
| :--- | :----------- |
| district_id | The unique identifier of the school district |
| state | The state where the district resides in |
| locale | NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See [Locale Boundaries User's Manual](https://eric.ed.gov/?id=ED577162) for more information. |
| pct_black/hispanic | Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data |
| pct_free/reduced | Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data |
| county_connections_ratio | `ratio` (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See [FCC data](https://www.fcc.gov/form-477-county-data-internet-access-services) for more information. |
| pp_total_raw | Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district. |


In [ ]:
districts_df = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')
districts_df.head()

### Engagement data
The engagement data are aggregated at school district level, and each file in the folder `engagement_data` represents data from one school district. The 4-digit file name represents `district_id` which can be used to link to district information in `district_info.csv`. The `lp_id` can be used to link to product information in `product_info.csv`.

| Name | Description |
| :--- | :----------- |
| time | date in "YYYY-MM-DD" |
| lp_id | The unique identifier of the product |
| pct_access | Percentage of students in the district have at least one page-load event of a given product and on a given day |
| engagement_index | Total page-load events per one thousand students of a given product and on a given day |


In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'

all_files = glob.glob(path + "/*.csv")

list_df = []
for file in all_files:
    df = pd.read_csv(file, header=0)
    df['district_id'] = file.split('/')[-1].split('.')[0]
    list_df.append(df)
    
engagement_df = pd.concat(list_df)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df.head()

In [ ]:
products_df.head()

In [ ]:
def Column_counts(df):
    counts = []
    for col in df.columns:
        counts.append(df[col].count())
        
    return counts

counts = Column_counts(products_df)        
px.bar(x=products_df.columns, y=counts, title='Products Data')

In [ ]:
counts = Column_counts(districts_df)        
px.bar(x=districts_df.columns, y=counts, title = "Districts Data")

In [ ]:
counts = Column_counts(engagement_df)        
px.bar(x=engagement_df.columns, y=counts, title='Engagement Data')

In [ ]:
px.pie(products_df, names='Sector(s)', title='Sectors Distribution')

In [ ]:
plt.figure(figsize=(8, 10))
sns.countplot(y='state', data=districts_df, order=districts_df['state'].value_counts().index)
plt.title('States Distribution')
plt.show()

In [ ]:
px.pie(districts_df, names='locale', title='locale Distribution')

In [ ]:
group_districts = engagement_df.groupby('district_id', as_index=False,dropna=True).mean()
districts_df = districts_df.sort_values(by='district_id').reset_index(drop=True)
districts_df['pct_access'] = group_districts['pct_access']
districts_df['engagement_index'] = group_districts['engagement_index']
districts_df.head()

In [ ]:
px.bar(districts_df, x='state', y='pct_access', title='State wise percentage access')

In [ ]:
px.bar(districts_df, x='state', y='engagement_index', title='State wise engagement index')

In [ ]:
df = districts_df.dropna()
px.sunburst(df, path=['locale', 'state'], values='engagement_index', title='Distribution of engagement index by state and locale', width=600, height=600)

In [ ]:
px.sunburst(df, path=['locale', 'state'], values='pct_access', title='Distribution of pct access by state and locale', width=600, height=600)

In [ ]:
df = engagement_df.groupby('time', as_index=False).mean()
df.head()

In [ ]:
px.line(df, x='time', y='pct_access', height=400, width=800,title='Percentage Access by time')

In [ ]:
px.line(df, x='time', y='engagement_index', height=400, width=800,title='Engagement index by time')

In [ ]:
# To Be Continued